In [1]:
import pandas as pd
import warnings
import re

In [2]:
def read_data ():
    data = pd.read_csv('results\EntrevistaInicial.csv')
    return data

In [3]:
def dicts ():
    dict_sust = {'Tabaco':(1,2) , 'Alcohol':(3,4,5,84), 'Marihuana':(6,7,8,9,10,11),'Inalables':(24,25,26,27,28) , 'Cocaina':(12,13,14,15,86) , 'Metanfetaminas': (16,17,18,19,85) , 'OtrosEstimulantes':(19,20,21,22,23), 'Extasis':(22,50,51,48), 'Benzodiacepinas': (52,53) , 'OtrosDepresores': (54,55,56), 'Alucinogenos': (29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47), 'Heroina': (60,61), 'OpiodesSinteticos': (62,63,64,65,66,67), 'OpioDerivados': (57,58,59), 'UtilidadMedica': (68,69,70,71,72,73,74,75), 'SustanciasDeAbuso': (76,77,78,79,80,81,82), 'Otros': (83,10000)}
    return dict_sust

In [4]:
def mod_sustancias (df, dict_sust):
    for col in df.columns:
        if col.startswith('SustanciaI'):
            for ind, val in  df [col].items():
                for key, value in dict_sust.items():
                    if val in value:
                        df.at[ind, col] = key
    return df


In [5]:
def dropcolumns (df):
    listg = []
    for col in df.columns:
        if col.startswith('SustanciaI') or col.startswith('FolioI') or col.startswith('OrdenConsumo'):
            listg.append(col)
        else:
            df = df.drop(columns = col)
    return df

In [6]:
def searchind(df, col):
    if col.startswith('SustanciaI'):
        exp = re.search(r'(SustanciaId)(\d)', col)
        if exp:
            return exp.group(2)
        else:
            return None

In [7]:
def sustancias1 (df, sust):
    list_folio = []
    for col in df.columns:
        for ind, val in df[col].items():
            if val == sust:
                exp = searchind(df, col)
                if df['OrdenConsumo'+str(exp)][ind] == 1:
                    list_folio.append(df.at[ind, 'FolioId'])
    return list_folio


In [8]:
def sustancias2 (df,sust):
    list_folio = []
    for col in df.columns:
        for ind, val in df[col].items():
            if val == sust:
                exp = searchind(df, col)
                if df['OrdenConsumo'+str(exp)][ind] == 2:
                    list_folio.append(df.at[ind, 'FolioId'])
    return list_folio   


In [9]:
def sustancias3 (df,sust):
    list_folio = []
    for col in df.columns:
        for ind, val in df[col].items():
            if val == sust:
                exp = searchind(df, col)
                if df['OrdenConsumo'+str(exp)][ind] == 3:
                    list_folio.append(df.at[ind, 'FolioId'])
    return list_folio  

In [10]:
def snkgr(df, list_sust, sust):
    dict_sust = {}
    #for key, val in list_sust.items():
    list = sustancias1(df, sust)
    df1 = df[df['FolioId'].isin(list)]
    for key, val in list_sust.items():
        list = sustancias2(df1, key)
        df2 = df1[df1['FolioId'].isin(list)]
        dict_sust[key] = (df2)
    return dict_sust

In [11]:
def dropcol(df):
    df['SustanciaId0'] = df['SustanciaId']
    df['OrdenConsumo0'] = df['OrdenConsumo']
    df.drop(columns = 'SustanciaId', inplace = True)
    df.drop(columns = 'OrdenConsumo', inplace = True)
    return df

In [12]:
def gen_dataset (list_sust):
    count = 0
    dict_aux = {0:'Tabaco', 1: 'Alcohol', 2: 'Marihuana', 3: 'Inalables', 4: 'Cocaina', 5: 'Metanfetaminas', 6: 'OtrosEstimulantes', 7: 'Extasis', 8: 'Benzodiacepinas', 9: 'OtrosDepresores', 10: 'Alucinogenos', 11: 'Heroina', 12: 'OpiodesSinteticos', 13: 'OpioDerivados', 14: 'UtilidadMedica', 15: 'SustanciasDeAbuso', 16: 'Otros'} 
    df_sankey1_2 = pd.DataFrame(columns = ['From', 'To', 'Value'])
    for i in range (0,16):
        for key, val in list_sust[i].items():
            aux = dict_aux[i]
            df_sankey1_2.loc[count] = [aux+str(1), key+str(2), len(val)]
            count += 1
    return df_sankey1_2

In [13]:
def main():
    list_sust = []
    df = read_data()
    df = dropcol(df)
    df = mod_sustancias(df, dicts())
    df = dropcolumns(df)
    sust = dicts()
    for key, val in sust.items():
        list_sust.append(snkgr(df, dicts(), key))
    df_sankey1_2 = gen_dataset(list_sust)
    return df_sankey1_2, list_sust        

In [14]:
warnings.filterwarnings("ignore")
df_sankey1_2, list_sust = main()

In [15]:
def snkgr3(df, list_sust, sust):
    dict_sust = {}
    list3 = []
    for droga in list_sust:
        list = sustancias3(df, droga)
        df2 = df[df['FolioId'].isin(list)]
        dict_sust[droga] = (df2)
        list3.append(dict_sust)
    return list3

In [16]:
listnvl3 = []
for i in range(0,16):
    for key, val in list_sust[i].items():
        listnvl3.append(snkgr3(list_sust[i][key], dicts(), key))


In [17]:
def gen_dataset2 (listnvl3):
    count = 0
    dict_aux = {0:'Tabaco', 1: 'Alcohol', 2: 'Marihuana', 3: 'Inalables', 4: 'Cocaina', 5: 'Metanfetaminas', 6: 'OtrosEstimulantes', 7: 'Extasis', 8: 'Benzodiacepinas', 9: 'OtrosDepresores', 10: 'Alucinogenos', 11: 'Heroina', 12: 'OpiodesSinteticos', 13: 'OpioDerivados', 14: 'UtilidadMedica', 15: 'SustanciasDeAbuso', 16: 'Otros'} 
    df_sankey2_3 = pd.DataFrame(columns = ['From', 'To', 'Value'])
    for i in range (0,16):
        for j in range(0,16):
            for key, val in listnvl3[i][j].items():
                aux = dict_aux[i]
                df_sankey2_3.loc[count] = [aux+str(2), key+str(3), len(val)]
                count += 1
    return df_sankey2_3

In [18]:
def moddf (df):
    dfnew = pd.concat([df.iloc[0:17], df.iloc[272:289], df.iloc[544:561], df.iloc[816:833], df.iloc[1088:1105], df.iloc[1360:1377], df.iloc[1632:1649], df.iloc[1904:1921], df.iloc[2176:2193], df.iloc[2448:2465], df.iloc[2720:2737], df.iloc[2992:3009], df.iloc[3264:3281], df.iloc[3536:3553], df.iloc[3808:3825], df.iloc[4080:4097]])
    return dfnew

In [19]:
df_sankey2_3 = gen_dataset2(listnvl3)
df_sankey2_3 = moddf(df_sankey2_3)
df_sankey = pd.concat([df_sankey1_2, df_sankey2_3])


In [20]:
df_sankey.to_csv('results\Sankey.csv', index = False)

In [21]:
import plotly.graph_objects as go

df = pd.read_csv('results\Sankey.csv')


labels = list(set(df['From']).union(set(df['To'])))
source = [labels.index(x) for x in df['From']]
target = [labels.index(x) for x in df['To']]
value = df['Value'].tolist()


fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels
    ),
    link=dict(
        source=source,
        target=target,
        value=value
    )
)])


fig.show()